In [1]:
# import packages

import os # read system path 
import csv

import matplotlib as mpl
import matplotlib.pyplot as plt
import librosa
import librosa.display

import pandas as pd
from scipy import signal
import soundfile as sf
from gudhi.point_cloud import timedelay
import numpy as np
from numpy import argmax
import math
from ripser import ripser
from persim import plot_diagrams

from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

import umap
from sklearn.preprocessing import StandardScaler
%matplotlib qt5

# Path is where the voiced/voicedless wav file located
voicedPath="/Users/pfeng3/Documents/research/TopCap/revise3/LJSpeech_seg_add500/voiced/"
voicedlessPath="/Users/pfeng3/Documents/research/TopCap/revise3/LJSpeech_seg_add500/voiceless/"

In [95]:
for fn in os.listdir(voicedPath):
    # Subsample dataset, retrieve 1 in 10 among dataset
    randNum=np.random.randint(10)
    if randNum !=0:
        continue

    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)
    
    # Write result in a csv file
    with open("MFCC_Diag3.csv","a",newline="") as csvfile:
        writer=csv.writer(csvfile)

        # MFCC
        mfccs = librosa.feature.mfcc(y=sig, sr=samplerate, n_mfcc=10, n_fft=int(len(sig)/2),hop_length=int(len(sig)/4))
        mfccs=np.abs(mfccs.flatten())

        # Add last feature to indicate if it is voiced/ voicedless
        # 0 indicate the phone is voiced
        data=np.append(mfccs,0)
 
        writer.writerow(data)

# For voicedless data
for fn in os.listdir(voicedlessPath):
    # Subsample dataset, retrieve 1 in 10 among dataset
    randNum=np.random.randint(10)
    if randNum !=0:
        continue
    
    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedlessPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)

    # Write result in a csv file
    with open("MFCC_Diag3.csv","a",newline="") as csvfile:
        writer=csv.writer(csvfile)

        # MFCC
        mfccs = librosa.feature.mfcc(y=sig, sr=samplerate, n_mfcc=10, n_fft=int(len(sig)/2),hop_length=int(len(sig)/4))
        mfccs=np.abs(mfccs.flatten())

        # Add last feature to indicate if it is voiced/ voicedless
        # 1 indicate the phone is voicedless
        data=np.append(mfccs,1)
 
        writer.writerow(data)


/opt/anaconda3/lib/python3.12/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [96]:
# Read the csv file into DataFrame
df=pd.read_csv('MFCC_Diag3.csv',header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,474.413680,394.702578,392.120166,334.495751,286.862525,35.675270,78.544135,72.156839,73.759740,103.862011,...,25.813745,34.500413,42.421679,21.884744,12.369543,7.273538,1.677957,0.847221,0.083038,0.0
1,434.973747,478.955935,489.708818,504.379999,406.492021,141.272132,139.233509,154.052982,161.168971,133.640820,...,28.160545,26.362843,19.528827,22.683410,3.455542,0.302308,3.533694,6.449537,10.890118,0.0
2,473.444847,488.837979,461.224367,351.784580,106.600163,200.821576,183.125104,175.407412,126.065529,89.177378,...,25.474883,17.245023,13.747942,18.467241,26.544052,37.008865,33.772867,24.422210,17.248702,0.0
3,451.464241,527.908819,541.824345,564.812681,487.843676,170.435578,188.693819,197.317286,186.555843,124.956792,...,17.377626,28.782961,18.844194,7.637665,12.420811,23.826020,25.151215,26.271143,6.956363,0.0
4,394.019158,427.919339,364.087257,376.223614,500.677735,169.708879,228.275038,189.156367,176.352522,132.893554,...,26.174388,16.409029,35.184183,10.896636,9.520860,26.161302,21.197071,22.997972,3.968198,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31096,295.721555,388.833466,408.163876,483.897974,542.505713,113.476406,112.996743,116.156079,127.846263,131.575824,...,1.755822,4.233910,0.799130,22.090053,8.007550,0.639300,6.203666,0.187366,13.833055,1.0
31097,401.872317,500.598333,443.259705,376.416790,530.807370,116.575883,121.590205,6.842608,62.303253,2.635672,...,6.070134,5.422245,0.479636,9.583966,3.091269,5.720183,9.507272,4.912721,4.304591,1.0
31098,451.176300,486.508965,377.603117,363.403345,362.487109,75.543008,79.075747,32.079732,49.135021,62.460621,...,16.047420,34.361024,9.678161,21.312363,14.243023,1.059416,4.644041,18.320143,1.739075,1.0
31099,349.078619,309.781445,333.054052,326.294891,356.328406,50.785772,47.954581,116.074710,116.000266,102.309120,...,24.798080,13.008676,11.945057,4.690728,3.827010,3.978565,2.045769,7.501623,3.118001,1.0


In [97]:
# Shuffle 9000 samples for each voice and voiceless type
n=50
# Shuffle 9000 rows from each class
df_0 = df[df[n] == 0].sample(n=9000, random_state=42)
df_1 = df[df[n] == 1].sample(n=9000, random_state=42)

# Combine and shuffle again (optional)
df = pd.concat([df_0, df_1]).sample(frac=1, random_state=42).reset_index(drop=True)

In [98]:
# Use UMAP to reduce the feature dimension to 2
reducer = umap.UMAP()
data = df.iloc[:,0:n]
scaled_data = StandardScaler().fit_transform(data)
embedding = reducer.fit_transform(scaled_data)
embedding.shape

(18000, 2)

In [99]:
# Read the embedded feature as DataFrame
df_feature=pd.DataFrame(embedding,columns=['feature1','feature2'])
df_feature['type']=df[n]
df_feature

,feature1,feature2,type
0,4.002078,2.348708,0.0
1,1.466092,5.552018,0.0
2,2.683566,2.301844,1.0
3,3.804989,5.437905,0.0
4,2.372371,2.617243,0.0
...,...,...,...
17995,4.560303,2.143786,1.0
17996,4.923023,1.202337,1.0
17997,-0.604417,8.925669,0.0
17998,-1.672662,2.330016,0.0


In [100]:
# Normalization: min-max
normalized_df=(df_feature[['feature1','feature2']]-df_feature[['feature1','feature2']].min())/(df_feature[['feature1','feature2']].max()-df_feature[['feature1','feature2']].min())
normalized_df['type']=df_feature['type']

In [ ]:
# Read/load data if needed
#normalized_df.to_pickle('mfcc_df_feature_nor')
#normalized_df=pd.read_pickle('mfcc_df_feature_nor')

In [3]:
# Check the number of samples in each class
(normalized_df['type']==1).sum()

9000

In [ ]:
# Plot the normalized result

# Set up plot configuration
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels

# Group the data based on voiced/ voicedless
groups = normalized_df.groupby('type')

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.margins(0.05)
typeDict= {1:'voiceless',0:'voiced'}
for type, group in groups:
    if type==1:
        ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type],alpha=0.5, color='#4d4dff')
    if type==0:
        ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type],alpha=0.5, color='#ff5c33')
legend=ax.legend(fontsize=15,markerscale=4,loc='upper right')
plt.xlabel('UMAP_1',fontsize=15)
plt.ylabel('UMAP_2',fontsize=15)

## Save figure as pdf file
#plt.savefig("/Users/pfeng3/Documents/research/TopCap/picture/featureAna_mfcc1.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# Plot individual 
plt.figure(figsize=(6, 3))

for type, group in groups:
    if type==0:
        plt.subplot(1, 2, 1)
        plt.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type], alpha=0.5, color='#ff5c33')
        plt.legend(['voiced'],fontsize=10,markerscale=4,loc='upper right')
        plt.xlabel('UMAP_1')
        plt.ylabel('UMAP_2')
        plt.xlim([-0.1,1.1])
        plt.ylim([-0.1,1.1])
        plt
    if type==1:
        plt.subplot(1, 2, 2)
        plt.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type], alpha=0.5, color='#4d4dff')
        plt.legend(['voiceless'],fontsize=10,markerscale=4,loc='upper right')
        plt.xlabel('UMAP_1')
        plt.ylabel('UMAP_2')
        plt.xlim([-0.1,1.1])
        plt.ylim([-0.1,1.1])

plt.tight_layout()
plt.show()

## Save figure as pdf file
#plt.savefig("/Users/pfeng3/Documents/research/TopCap/picture/featureAna_mfcc2.pdf", format="pdf", bbox_inches="tight")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df[['feature1','feature2']], normalized_df['type'], test_size=0.2, random_state=42)

In [36]:
# Classification model knn 

knn = KNeighborsClassifier(n_neighbors=20) # Set the number of neighbors
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7972222222222223


In [108]:
# Train a linear model to classify the data, and plot the decision boundary

# Create a RidgeClassifier object
ridge = RidgeClassifier(alpha=1.0)

# Fit the model to the training data
ridge.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = ridge.predict(X_test)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8836111111111111


In [18]:
# Create a meshgrid to plot decision boundary
x_min, x_max = X['feature1'].min() - 1, X['feature1'].max() + 1
y_min, y_max = X['feature2'].min() - 1, X['feature2'].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

# Predict the class labels for each point in the grid
Z = ridge.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the data points
fig, ax = plt.subplots()
ax.margins(0.05)
typeDict= {1:'voicedless',0:'voiced'}
for type, group in groups:
    ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type])
ax.legend()
plt.xlabel('feature1')
plt.ylabel('feature2')

# Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.8)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RidgeClassifier was fitted with feature names
  warnings.warn(


In [38]:
# SVM classification
from sklearn import svm

# Create an SVM classifier
clf = svm.SVC(kernel='linear')
# Train the classifier
clf.fit(X_train, y_train.values.ravel())
# Predict on the test set
y_pred = clf.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7416666666666667


In [4]:
# Logistic classification
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
model = LogisticRegression()

# Train the model using the training data
model.fit(X_train, y_train.values.ravel())

# Predict class labels for the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8847222222222222


In [40]:
# LDA model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
# Train the classifier
clf.fit(X_train, y_train.values.ravel())
# Predict on the test set
y_pred = clf.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7388888888888889


In [8]:
# compute statistical information from data

normalized_df['feature1'].mean()
normalized_df['feature1'].std()

0.28596314787864685

In [10]:
normalized_df['feature2'].mean()
normalized_df['feature2'].std()

0.2480863332748413

In [12]:
nor_voice_df=normalized_df[normalized_df['type']==0]
print(f'Mean for voiced data: {nor_voice_df.mean()}')
print(f'Std for voiced data: {nor_voice_df.std()}')

Mean for voiced data: feature1    0.802878
feature2    0.479979
type        0.000000
dtype: float64
Std for voiced data: feature1    0.125124
feature2    0.243427
type        0.000000
dtype: float64


In [16]:
nor_voiceless_df=normalized_df[normalized_df['type']==1]
print(f'Mean for voiceless data: {nor_voiceless_df.mean()}')
print(f'Std for voiceless data: {nor_voiceless_df.std()}')

Mean for voiceless data: feature1    0.339149
feature2    0.499122
type        1.000000
dtype: float64
Std for voiceless data: feature1    0.210309
feature2    0.253263
type        0.000000
dtype: float64
